In [52]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import spacy
import gensim
import operator
import re
import string


In [53]:
df = pd.read_csv('movies.csv')
df.head()

,rank,title,genre,wiki_plot,imdb_plot
0,0,The Godfather,"[u' Crime', u' Drama']","On the day of his only daughter's wedding, Vit...","In late summer 1945, guests are gathered for t..."
1,1,The Shawshank Redemption,"[u' Crime', u' Drama']","In 1947, banker Andy Dufresne is convicted of ...","In 1947, Andy Dufresne (Tim Robbins), a banker..."
2,2,Schindler's List,"[u' Biography', u' Drama', u' History']","In 1939, the Germans move Polish Jews into the...",The relocation of Polish Jews from surrounding...
3,3,Raging Bull,"[u' Biography', u' Drama', u' Sport']","In a brief scene in 1964, an aging, overweight...","The film opens in 1964, where an older and fat..."
4,4,Casablanca,"[u' Drama', u' Romance', u' War']",It is early December 1941. American expatriate...,"In the early years of World War II, December 1..."


In [54]:
from spacy.lang.en.stop_words import STOP_WORDS
spacy_nlp = spacy.load('en_core_web_sm')
punctuations = string.punctuation
stop_words = spacy.lang.en.stop_words.STOP_WORDS

In [55]:
def spacy_tokenizer(sentence):
 
    #remove distracting single quotes
    sentence = re.sub('\'','',sentence)

    #remove digits adnd words containing digits
    sentence = re.sub('\w*\d\w*','',sentence)

    #replace extra spaces with single space
    sentence = re.sub(' +',' ',sentence)

    #remove unwanted lines starting from special charcters
    sentence = re.sub(r'\n: \'\'.*','',sentence)
    sentence = re.sub(r'\n!.*','',sentence)
    sentence = re.sub(r'^:\'\'.*','',sentence)
    
    #remove non-breaking new line characters
    sentence = re.sub(r'\n',' ',sentence)
    
    #remove punctunations
    sentence = re.sub(r'[^\w\s]',' ',sentence)
    
    #creating token object
    tokens = spacy_nlp(sentence)
    
    #lower, strip and lemmatize
    tokens = [word.lemma_.lower().strip() if word.lemma_ != "-PRON-" else word.lower_ for word in tokens]
    
    #remove stopwords, and exclude words less than 2 characters
    tokens = [word for word in tokens if word not in stop_words and word not in punctuations and len(word) > 2]
    
    #return tokens
    return tokens


In [56]:
df['tokenized_wiki']= df ['wiki_plot'].map(lambda x: spacy_tokenizer(x))

In [57]:
df.head()

,rank,title,genre,wiki_plot,imdb_plot,tokenized_wiki
0,0,The Godfather,"[u' Crime', u' Drama']","On the day of his only daughter's wedding, Vit...","In late summer 1945, guests are gathered for t...","[day, daughter, wedding, vito, corleone, hear,..."
1,1,The Shawshank Redemption,"[u' Crime', u' Drama']","In 1947, banker Andy Dufresne is convicted of ...","In 1947, Andy Dufresne (Tim Robbins), a banker...","[banker, andy, dufresne, convict, murder, wife..."
2,2,Schindler's List,"[u' Biography', u' Drama', u' History']","In 1939, the Germans move Polish Jews into the...",The relocation of Polish Jews from surrounding...,"[germans, polish, jews, kraków, ghetto, world,..."
3,3,Raging Bull,"[u' Biography', u' Drama', u' Sport']","In a brief scene in 1964, an aging, overweight...","The film opens in 1964, where an older and fat...","[brief, scene, age, overweight, italian, ameri..."
4,4,Casablanca,"[u' Drama', u' Romance', u' War']",It is early December 1941. American expatriate...,"In the early years of World War II, December 1...","[early, december, american, expatriate, rick, ..."


In [58]:
movie_plot= df['tokenized_wiki']
movie_plot[:3]

0    [day, daughter, wedding, vito, corleone, hear,...
1    [banker, andy, dufresne, convict, murder, wife...
2    [germans, polish, jews, kraków, ghetto, world,...
Name: tokenized_wiki, dtype: object

In [59]:
from gensim import corpora
dictionary = corpora.Dictionary(movie_plot)


In [66]:
stop_ids = [dictionary.token2id[stopword] for stopword in stoplist if stopword in dictionary.token2id]
dictionary.filter_tokens(stop_ids)

In [74]:
vals=[[val,key] for key,val in dictionary.items() if key<=50]
vals

[['abrasive', 0],
 ['abroad', 1],
 ['abuse', 2],
 ['accept', 3],
 ['accuse', 4],
 ['act', 5],
 ['adams', 6],
 ['add', 7],
 ['address', 8],
 ['agree', 9],
 ['ambush', 10],
 ['angeles', 11],
 ['answer', 12],
 ['anthony', 13],
 ['apollonia', 14],
 ['arrange', 15],
 ['ask', 16],
 ['assassin', 17],
 ['assassination', 18],
 ['associate', 19],
 ['attack', 20],
 ['attempt', 21],
 ['authority', 22],
 ['aware', 23],
 ['baron', 24],
 ['barzini', 25],
 ['battle', 26],
 ['bed', 27],
 ['betrayal', 28],
 ['bodyguard', 29],
 ['bomb', 30],
 ['booth', 31],
 ['brasi', 32],
 ['brasis', 33],
 ['break', 34],
 ['bronx', 35],
 ['brother', 36],
 ['business', 37],
 ['buy', 38],
 ['capos', 39],
 ['captain', 40],
 ['car', 41],
 ['career', 42],
 ['carlo', 43],
 ['casino', 44],
 ['christening', 45],
 ['christmas', 46],
 ['clampdown', 47],
 ['clemenza', 48],
 ['collapse', 49],
 ['come', 50]]

[['abrasive', 0],
 ['abroad', 1],
 ['abuse', 2],
 ['accept', 3],
 ['accuse', 4],
 ['act', 5],
 ['adams', 6],
 ['add', 7],
 ['address', 8],
 ['agree', 9],
 ['ambush', 10],
 ['angeles', 11],
 ['answer', 12],
 ['anthony', 13],
 ['apollonia', 14],
 ['arrange', 15],
 ['ask', 16],
 ['assassin', 17],
 ['assassination', 18],
 ['associate', 19],
 ['attack', 20],
 ['attempt', 21],
 ['authority', 22],
 ['aware', 23],
 ['baron', 24],
 ['barzini', 25],
 ['battle', 26],
 ['bed', 27],
 ['betrayal', 28],
 ['bodyguard', 29],
 ['bomb', 30],
 ['booth', 31],
 ['brasi', 32],
 ['brasis', 33],
 ['break', 34],
 ['bronx', 35],
 ['brother', 36],
 ['business', 37],
 ['buy', 38],
 ['capos', 39],
 ['captain', 40],
 ['car', 41],
 ['career', 42],
 ['carlo', 43],
 ['casino', 44],
 ['christening', 45],
 ['christmas', 46],
 ['clampdown', 47],
 ['clemenza', 48],
 ['collapse', 49],
 ['come', 50]]